# Java Persistence API (JPA)


JPA est une [spécification (actuellement en version 3.1)](https://jakarta.ee/specifications/persistence/3.1/jakarta-persistence-spec-3.1.html) concernant la liaison entre le modèle orienté objet de Java et le modèle relationnel (Object Relational Mapping - ORM). JPA est construit au dessus de [JDBC](https://docs.oracle.com/javase/8/docs/technotes/guides/jdbc/)

In [4]:
%maven jakarta.persistence:jakarta.persistence-api:3.1.0

JPA possède trois parties principales : 
 * La description des "entités" persistentes.
 * L'interaction de base (CRUD) avec la persistence.
 * L'interrogation native ou via un langage dédié.
 
 
JPA génère donc automatiquement les requêtes SQL et leur mise en oeuvre avec JDBC. 

Plusieurs implantations de JPA existent dont :

  * EclipseLink 4.0.0-M3
  

In [5]:
%maven org.eclipse.persistence:eclipselink:4.0.0-M3

  * Hibernate ORM 6.0.0.Final
  
`org.hibernate.orm:hibernate-core:6.1.6.Final`

## Les entités

In [6]:
%%loadFromPOM 
  <dependency>
    <groupId>jakarta.persistence</groupId>
    <artifactId>jakarta.persistence-api</artifactId>
    <version>3.0.0</version>
  </dependency>
  <dependency>
    <groupId>org.eclipse.persistence</groupId>
    <artifactId>eclipselink</artifactId>
    <version>3.0.3</version>
  </dependency>

In [2]:
%%compile fr/univtln/bruno/Location.java

package fr.univtln.bruno;

import jakarta.persistence.*;
import lombok.*;           

@Setter
@Getter         
@ToString
@AllArgsConstructor(staticName="of")
@NoArgsConstructor
@EqualsAndHashCode(onlyExplicitlyIncluded = true)

@Entity @Table( name="LOCATION" )
public class Location {

 @EqualsAndHashCode.Include
 @Column( name="ID" ) 
 @Id
 private String id;
    
 @Column( name="DESCRIPTION" ) 
 private String description;
    
 @PrePersist
 public void logNewLocationAttempt() {
    System.out.println("Attempting to add new location: " + this);
 }     
     
 @PostPersist
 public void logNewLocationAdded() {
    System.out.println("Added new location: " + this);     
 }
}

In [3]:
%%compile fr/univtln/bruno/Sensor.java

package fr.univtln.bruno;

import jakarta.persistence.*;
import lombok.*;           

@Setter
@Getter         
@ToString
@AllArgsConstructor(staticName="of")
@NoArgsConstructor
@EqualsAndHashCode(onlyExplicitlyIncluded = true)

@Entity @Table( name="SENSOR" )
public class Sensor {

 public enum Type {CVC_TH, CVC_P, CVC_H, MO};
    
 @EqualsAndHashCode.Include
 @Column( name="ID" ) 
 @Id
 @GeneratedValue
 private long id;
     
 @Column( name="TYPE")     
 private Type type;

 @ManyToOne
 @JoinColumn( name="LOCATION" ) 
 private Location location;
}

In [4]:
import java.util.stream.Stream;
import fr.univtln.bruno.Location;
Map<String, Location> locations = Stream.of(new String[][] { 
    {"E00","Cuisine"},
    {"E01","Salon"},
    {"E02","Salle à manger"},
    {"E03","Chambre"}
    }).collect(Collectors.collectingAndThen(
        Collectors.toMap(data -> data[0], data -> Location.of(data[0], data[1])), 
        Collections::<String, Location> unmodifiableMap));

locations;

{E00=Location(id=E00, description=Cuisine), E02=Location(id=E02, description=Salle à manger), E01=Location(id=E01, description=Salon), E03=Location(id=E03, description=Chambre)}

## Le gestionnaire d'entités
L'entity manager

In [5]:
%jars h2*.jar
import org.h2.tools.Server;
import java.sql.SQLException;
Server h2Server;
try {
        h2Server = Server.createTcpServer(new String[]{"-ifNotExists"}).start();
        if (h2Server.isRunning(true)) {
            System.out.println(h2Server.getStatus());
        } else {
            throw new RuntimeException("Could not start H2 server.");
        }
    } catch (SQLException e) {
        throw new RuntimeException("Failed to start H2 server: ", e);
    }

TCP server running at tcp://172.17.0.2:9092 (only local connections)


In [6]:
%jars persistence.jar
%jars h2*.jar
    
import jakarta.persistence.EntityManager;
import jakarta.persistence.EntityManagerFactory;
import jakarta.persistence.EntityTransaction;
import jakarta.persistence.Persistence;

EntityManagerFactory entityManagerFactory = Persistence.createEntityManagerFactory("sensors_pu");

[EL Info]: 2023-01-20 10:38:33.076--ServerSession(1903532207)--EclipseLink, version: Eclipse Persistence Services - 3.0.3.v202208191135
[EL Fine]: sql: 2023-01-20 10:38:33.19--ServerSession(1903532207)--Connection(1665435422)--ALTER TABLE SENSOR DROP CONSTRAINT FK_SENSOR_LOCATION
[EL Fine]: sql: 2023-01-20 10:38:33.192--ServerSession(1903532207)--SELECT 1
[EL Fine]: sql: 2023-01-20 10:38:33.195--ServerSession(1903532207)--Connection(1665435422)--DROP TABLE LOCATION
[EL Fine]: sql: 2023-01-20 10:38:33.196--ServerSession(1903532207)--Connection(1665435422)--DROP TABLE SENSOR
[EL Fine]: sql: 2023-01-20 10:38:33.198--ServerSession(1903532207)--SELECT 1
[EL Fine]: sql: 2023-01-20 10:38:33.199--ServerSession(1903532207)--Connection(1665435422)--DROP TABLE SENSOR
[EL Fine]: sql: 2023-01-20 10:38:33.199--ServerSession(1903532207)--SELECT 1
[EL Fine]: sql: 2023-01-20 10:38:33.2--ServerSession(1903532207)--Connection(1665435422)--DROP TABLE SENSOR
[EL Fine]: sql: 2023-01-20 10:38:33.2--ServerSes

In [7]:
EntityManager entityManager = entityManagerFactory.createEntityManager();

entityManager.getTransaction().begin();
locations.values().stream().forEach(l->entityManager.persist(l));
entityManager.getTransaction().commit();

entityManager.close();

Attempting to add new location: Location(id=E00, description=Cuisine)
Attempting to add new location: Location(id=E02, description=Salle à manger)
Attempting to add new location: Location(id=E01, description=Salon)
Attempting to add new location: Location(id=E03, description=Chambre)
[EL Fine]: sql: 2023-01-20 10:38:33.497--ClientSession(1541892169)--Connection(1665435422)--INSERT INTO LOCATION (ID, DESCRIPTION) VALUES (?, ?)
	bind => [E00, Cuisine]
Added new location: Location(id=E00, description=Cuisine)
[EL Fine]: sql: 2023-01-20 10:38:33.499--ClientSession(1541892169)--Connection(1665435422)--INSERT INTO LOCATION (ID, DESCRIPTION) VALUES (?, ?)
	bind => [E01, Salon]
Added new location: Location(id=E01, description=Salon)
[EL Fine]: sql: 2023-01-20 10:38:33.499--ClientSession(1541892169)--Connection(1665435422)--INSERT INTO LOCATION (ID, DESCRIPTION) VALUES (?, ?)
	bind => [E03, Chambre]
Added new location: Location(id=E03, description=Chambre)
[EL Fine]: sql: 2023-01-20 10:38:33.50

In [8]:
EntityManager entityManager = entityManagerFactory.createEntityManager();
Location location = entityManager.find(Location.class, "E00");
System.out.println(location);

Location(id=E00, description=Cuisine)


## Les identifiants

In [9]:
import fr.univtln.bruno.Sensor;

EntityManager entityManager = entityManagerFactory.createEntityManager();

List<Sensor> sensors = Arrays.asList(Sensor.of(0,Sensor.Type.CVC_TH, locations.get("E00")),
    Sensor.of(0,Sensor.Type.MO, locations.get("E00")),
    Sensor.of(0,Sensor.Type.MO, locations.get("E01")));

entityManager.getTransaction().begin();
sensors.forEach(entityManager::persist);
entityManager.getTransaction().commit();

entityManager.close();    

class fr.univtln.bruno.Sensor
class fr.univtln.bruno.Sensor
class fr.univtln.bruno.Sensor
[EL Fine]: sql: 2023-01-20 10:38:33.97--ClientSession(1709325021)--Connection(1665435422)--UPDATE SEQUENCE SET SEQ_COUNT = SEQ_COUNT + ? WHERE SEQ_NAME = ?
	bind => [50, SEQ_GEN]
[EL Fine]: sql: 2023-01-20 10:38:33.972--ClientSession(1709325021)--Connection(1665435422)--SELECT SEQ_COUNT FROM SEQUENCE WHERE SEQ_NAME = ?
	bind => [SEQ_GEN]
[EL Fine]: sql: 2023-01-20 10:38:34.013--ClientSession(1709325021)--Connection(1665435422)--INSERT INTO SENSOR (ID, TYPE, LOCATION) VALUES (?, ?, ?)
	bind => [3, 3, E01]
[EL Fine]: sql: 2023-01-20 10:38:34.015--ClientSession(1709325021)--Connection(1665435422)--INSERT INTO SENSOR (ID, TYPE, LOCATION) VALUES (?, ?, ?)
	bind => [2, 3, E00]
[EL Fine]: sql: 2023-01-20 10:38:34.016--ClientSession(1709325021)--Connection(1665435422)--INSERT INTO SENSOR (ID, TYPE, LOCATION) VALUES (?, ?, ?)
	bind => [1, 0, E00]


## Les relations entre entités

## L'héritage

## L'interrogation
JPQL

## Concurrence

## Avancé
### Contrôle des attributs
### Cycle de vie

In [14]:
%%shell 
java -cp h2*.jar org.h2.tools.Shell \
  -url "jdbc:h2:tcp://localhost/~/sensors;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE;DEFAULT_NULL_ORDERING=HIGH" \
  -user demoJPA -password demoJPA -sql "SHOW TABLES; SHOW COLUMNS FROM LOCATION; SELECT * FROM LOCATION"

table_name | table_schema
location   | public
sensor     | public
sequence   | public
(3 rows, 3 ms)
field       | type              | NULL | key | default
id          | CHARACTER VARYING | NO   | PRI | NULL
description | CHARACTER VARYING | YES  |     | NULL
(2 rows, 1 ms)
id  | description
E00 | Cuisine
E01 | Salon
E03 | Chambre
E02 | Salle à manger
(4 rows, 0 ms)


In [11]:
%%loadFromPOM
<dependency>
    <groupId>tech.tablesaw</groupId>
    <artifactId>tablesaw-jsplot</artifactId>
    <version>0.43.1</version>
</dependency>

In [15]:
import java.sql.Connection;
import java.sql.Statement;
import java.sql.DriverManager;
import java.sql.ResultSet;
import tech.tablesaw.api.Table;

String DB_URL = "jdbc:h2:tcp://localhost/~/sensors;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE;DEFAULT_NULL_ORDERING=HIGH";
Connection conn = DriverManager.getConnection(DB_URL,"demoJPA","demoJPA");

Table table = null; 
try (Statement stmt = conn.createStatement()) {
  String sql = "SELECT * FROM LOCATION";
  try (ResultSet results = stmt.executeQuery(sql)) {
    table = Table.read().db(results, "LOCATION");
  }
}

table;

         LOCATION         
 id   |   description    |
--------------------------
 E00  |         Cuisine  |
 E01  |           Salon  |
 E03  |         Chambre  |
 E02  |  Salle à manger  |

In [16]:
import java.sql.Connection;
import java.sql.Statement;
import java.sql.DriverManager;
import java.sql.ResultSet;
import tech.tablesaw.api.Table;

String DB_URL = "jdbc:h2:tcp://localhost/~/sensors;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE;DEFAULT_NULL_ORDERING=HIGH";
Connection conn = DriverManager.getConnection(DB_URL,"demoJPA","demoJPA");

Table table = null; 
try (Statement stmt = conn.createStatement()) {
  String sql = "SELECT * FROM SENSOR";
  try (ResultSet results = stmt.executeQuery(sql)) {
    table = Table.read().db(results, "SENSOR");
  }
}

table;

           SENSOR           
 id  |  type  |  location  |
----------------------------
  1  |     0  |       E00  |
  2  |     3  |       E00  |
  3  |     3  |       E01  |

In [13]:
//h2Server.stop();